In [21]:
!pip install git+https://github.com/openai/CLIP.git


  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-qm3fz0bh
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-req-build-qm3fz0bh
  Resolved https://github.com/openai/CLIP.git to commit a1d071733d7111c9c014f024669f959182114e33
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.4/53.4 kB 2.5 MB/s eta 0:00:00
  Created wheel for clip: filename=clip-1.0-py3-none-any.whl size=1369497 sha256=41f53af33e52b2319879e6fc30d4a2c181e7de0f6438bd36b50a7cb9fd4b5e77
  Stored in directory: /tmp/pip-ephem-wheel-cache-b8uoj5po/wheels/da/2b/4c/d6691fa9597aac8bb85d2ac13b112deb897d5b50f5ad9a37e4
Successfully built clip


In [19]:
!pip install openai


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.7/226.7 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.8/77.8 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 8.7 MB/s eta 0:00:00


In [8]:
!pip install torchvision


In [6]:
!pip install transformers


In [2]:
import requests
from PIL import Image
from io import BytesIO
import os
import pandas as pd



# Read the dataset
df = pd.read_csv('/content/fakenews_annotation.csv')

# Create a directory for images if it doesn't exist
image_folder = '/content/images'
if not os.path.exists(image_folder):
    os.makedirs(image_folder)

# Session to persist certain parameters and reuse TCP connection
session = requests.Session()
session.headers.update({
    'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:88.0) Gecko/20100101 Firefox/88.0'
})

def safe_filename(filename):
    """Generate a safe filename that is filesystem-friendly"""
    return "".join([c for c in filename if c.isalpha() or c.isdigit() or c in ' ._-']).rstrip()

def download_image(image_url, title, label, index, session):
    try:
        # Send a GET request to the image URL
        response = session.get(image_url, timeout=10)

        # If the response status code is 200 (OK), save the image
        if response.status_code == 200:
            # Open the image and save it
            image = Image.open(BytesIO(response.content))
            # Generate a safe filename by including the title and label
            safe_title = safe_filename(title)
            filename = f'image_{safe_title}_{label}_{index}.jpg'
            image_path = os.path.join(image_folder, filename)
            image.save(image_path)
            print(f'Downloaded image to {image_path}')
        else:
            print(f'Failed to download image at index {index} with status code: {response.status_code}')
    except requests.exceptions.RequestException as e:
        print(f'Request failed at index {index}: {e}')

# Download images using the URLs from the DataFrame
for index, row in df.iterrows():
    download_image(row['Image_url'], row['Title'], row['Label'], index, session)


Downloaded image to /content/images/image_Tucker Carlson warns Dems now totally totalitarian_1_0.jpg
Downloaded image to /content/images/image_Supremes slap down Bidens eviction moratorium_1_1.jpg
Downloaded image to /content/images/image_GOP stages COVID revolt as Biden resurrects masking_0_2.jpg
Downloaded image to /content/images/image_Canada reopens border to Americans but only for the vaccinated_1_3.jpg
Downloaded image to /content/images/image_Vaccine passports blasted as stealth move to tyranny_0_4.jpg
Downloaded image to /content/images/image_WSJ directly targets White Evangelicals as greatest evaders of COVID vaccine among religious groups_0_5.jpg
Downloaded image to /content/images/image_CNN admits one governor got it right on lockdown_0_6.jpg
Downloaded image to /content/images/image_4 Things I Wish I Had Known Before the SHTF in Venezuela_1_7.jpg
Downloaded image to /content/images/image_Biden and Pelosi missing the point about health care options_1_8.jpg
Downloaded image t

In [5]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
import requests
from PIL import Image
from io import BytesIO
import os

# NLTK downloads (do this once)
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('omw-1.4')

# Text preprocessing function
def preprocess_text(text):
    tokens = word_tokenize(text)
    tokens = [word.lower() for word in tokens if word.isalpha()]
    stop_words = set(stopwords.words('english'))
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens if not word in stop_words]
    return ' '.join(tokens)  # Return a string of the processed title

# Image downloading function
def download_image(image_url, title, label, index, session, image_folder):
    filename = f'image_{safe_filename(title)}_{label}_{index}.jpg'
    image_path = os.path.join(image_folder, filename)
    try:
        response = session.get(image_url, timeout=10)
        if response.status_code == 200:
            image = Image.open(BytesIO(response.content))
            image.save(image_path)
            return image_path  # Return the path where the image was saved
        else:
            print(f'Failed to download image at index {index} with status code: {response.status_code}')
            return None
    except requests.exceptions.RequestException as e:
        print(f'Request failed at index {index}: {e}')
        return None

# Ensure the image folder exists
image_folder = '/content/images'
os.makedirs(image_folder, exist_ok=True)

# Initialize a requests session
session = requests.Session()
session.headers.update({'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:88.0) Gecko/20100101 Firefox/88.0'})

# Load the dataset
df = pd.read_csv('/content/fakenews_annotation.csv')

# Apply text preprocessing and download images
df['Processed_Title'] = df['Title'].apply(preprocess_text)
df['Image_Path'] = df.apply(lambda row: download_image(row['Image_url'], row['Title'], row['Label'], row.name, session, image_folder), axis=1)

# Save the updated dataframe
df.to_csv('/content/processed_fakenews_data_with_images.csv', index=False)

# Display the first few rows to verify
print(df.head())


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


                                               Title  \
0  Tucker Carlson warns Dems now totally 'totalit...   
1     Supremes slap down Biden's eviction moratorium   
2  GOP stages COVID revolt as Biden resurrects ma...   
3  Canada reopens border to Americans, but only f...   
4  Vaccine passports blasted as 'stealth move to ...   

                                            News_url  \
0  https://www.wnd.com/2021/08/tucker-carlson-war...   
1  https://www.wnd.com/2021/08/supremes-slap-bide...   
2  https://www.wnd.com/2021/07/gop-stages-covid-r...   
3  https://www.wnd.com/2021/08/canada-reopens-bor...   
4  https://www.wnd.com/2021/05/vaccine-passports-...   

                                           Image_url  Label  \
0  https://www.wnd.com/wp-content/uploads/2021/04...      1   
1  https://www.wnd.com/wp-content/uploads/2020/10...      1   
2  https://www.wnd.com/wp-content/uploads/2020/05...      0   
3  https://www.wnd.com/wp-content/uploads/2019/07...      1   
4  https://

Feature Extraction:

Text: Extract features from preprocessed titles using a text embedding model like BERT.
Image: Extract features from images using a pretrained CNN or Vision Transformer (ViT).

In [7]:
from transformers import BertTokenizer, BertModel
import torch

# Initialize tokenizer and model from pre-trained BERT base model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

# Function to encode text data
def encode_text(text):
    # Tokenize the text and convert to input format expected by BERT
    encoded_input = tokenizer(text, padding=True, truncation=True, return_tensors='pt')
    # Move encoded input to the same device as the model
    encoded_input = encoded_input.to(model.device)
    # Forward pass, get the model output (ignore pooled outputs)
    with torch.no_grad():
        output = model(**encoded_input)
    # Return only the last hidden state
    return output.last_hidden_state

# Encode all processed titles
df['Text_Features'] = df['Processed_Title'].apply(lambda x: encode_text(x).cpu().numpy())

# Save the dataframe with the new text features
df.to_pickle('/content/processed_fakenews_data_with_text_features.pkl')

# Print the shape of the features for the first article
print(df.iloc[0]['Text_Features'].shape)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

(1, 8, 768)


 Prepare the Dataset for Training
 First, we need to prepare the data for training. This involves combining the text and image features into a single feature array for each news item and then splitting the dataset into training and testing sets.

In [10]:
from torchvision import models, transforms
from PIL import Image
import torch

# Check if CUDA is available and set the device accordingly
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Initialize the pre-trained ResNet model
resnet = models.resnet50(pretrained=True).to(device)
resnet.eval()  # Set the model to evaluation mode

# Define image transformations
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Function to encode image data
def encode_image(image_path):
    # Open image and apply transformations
    image = Image.open(image_path).convert('RGB')
    image_tensor = transform(image).unsqueeze(0)  # Add batch dimension
    # Move image tensor to the device
    image_tensor = image_tensor.to(device)
    # Forward pass, get the model output
    with torch.no_grad():
        features = resnet(image_tensor)
    # Return features as a numpy array
    return features.cpu().numpy()

# Encode all images
df['Image_Features'] = df['Image_Path'].apply(lambda x: encode_image(x))

# Save the dataframe with the new image features
df.to_pickle('/content/processed_fakenews_data_with_all_features.pkl')

# Print the shape of the features for the first image
print(df.iloc[0]['Image_Features'].shape)


(1, 1000)


In [13]:
# Check the shape of the first few text features to understand the inconsistency
for i, text_feat in enumerate(df['Text_Features'][:5]):
    print(f"Sample {i} text feature shape: {text_feat.shape}")

# Check the shape of the first few image features
for i, image_feat in enumerate(df['Image_Features'][:5]):
    print(f"Sample {i} image feature shape: {image_feat.shape}")


Sample 0 text feature shape: (1, 8, 768)
Sample 1 text feature shape: (1, 11, 768)
Sample 2 text feature shape: (1, 16, 768)
Sample 3 text feature shape: (1, 11, 768)
Sample 4 text feature shape: (1, 9, 768)
Sample 0 image feature shape: (1, 1000)
Sample 1 image feature shape: (1, 1000)
Sample 2 image feature shape: (1, 1000)
Sample 3 image feature shape: (1, 1000)
Sample 4 image feature shape: (1, 1000)


In [15]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

# Step 1: Prepare text features by averaging across tokens
df['Text_Features_Avg'] = df['Text_Features'].apply(lambda x: np.mean(x, axis=1).squeeze())


In [16]:
# Verify the new shape
print(df.iloc[0]['Text_Features_Avg'].shape)

(768,)


In [17]:
# Step 2: Combine text and image features into a single vector
# Note: Ensure the Image_Features column is correctly processed before this step
df['Combined_Features'] = df.apply(lambda row: np.concatenate([row['Text_Features_Avg'], row['Image_Features'].squeeze()]), axis=1)

# Extract combined features and labels
X = np.stack(df['Combined_Features'].values)
y = df['Label'].values

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Verify shapes
print(f"X_train shape: {X_train.shape}, y_train shape: {y_train.shape}")
print(f"X_test shape: {X_test.shape}, y_test shape: {y_test.shape}")

# Initialize and train the logistic regression model
model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)

# Predict on the testing set
y_pred = model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")
print(classification_report(y_test, y_pred))

X_train shape: (207, 1768), y_train shape: (207,)
X_test shape: (52, 1768), y_test shape: (52,)
Accuracy: 0.5576923076923077
              precision    recall  f1-score   support

           0       0.50      0.57      0.53        23
           1       0.62      0.55      0.58        29

    accuracy                           0.56        52
   macro avg       0.56      0.56      0.56        52
weighted avg       0.56      0.56      0.56        52



In [18]:
import torch
import torch.nn as nn
import torch.optim as optim

class MultimodalClassifier(nn.Module):
    def __init__(self, text_feature_dim, image_feature_dim, hidden_dim, output_dim):
        super(MultimodalClassifier, self).__init__()
        self.fc1 = nn.Linear(text_feature_dim + image_feature_dim, hidden_dim)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_dim, output_dim)
        self.sigmoid = nn.Sigmoid()  # Use sigmoid for binary classification

    def forward(self, text_features, image_features):
        combined_features = torch.cat((text_features, image_features), dim=1)
        x = self.fc1(combined_features)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.sigmoid(x)
        return x

# Example initialization
model = MultimodalClassifier(text_feature_dim=768, image_feature_dim=1000, hidden_dim=512, output_dim=1)
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.BCELoss()  # Binary Cross-Entropy Loss for binary classification


CLIP for feature extraction, given its strong performance on various tasks. You can use the embeddings from CLIP as features for a simpler classifier like a neural network or even directly for your task with some fine-tuning.

In [23]:
import clip
from PIL import Image
import torch

# Ensure you're using the GPU provided by Colab Pro
device = "cuda" if torch.cuda.is_available() else "cpu"

# Load the CLIP model
clip_model, preprocess = clip.load("ViT-B/32", device=device)


100%|███████████████████████████████████████| 338M/338M [00:22<00:00, 16.0MiB/s]


In [24]:
import pandas as pd

# Load the dataset that contains both the titles and image paths
df = pd.read_csv('/content/processed_fakenews_data_with_images.csv')

print(df.head())  # To verify that it contains the columns 'Title' and 'Image_Path' or their equivalents


                                               Title  \
0  Tucker Carlson warns Dems now totally 'totalit...   
1     Supremes slap down Biden's eviction moratorium   
2  GOP stages COVID revolt as Biden resurrects ma...   
3  Canada reopens border to Americans, but only f...   
4  Vaccine passports blasted as 'stealth move to ...   

                                            News_url  \
0  https://www.wnd.com/2021/08/tucker-carlson-war...   
1  https://www.wnd.com/2021/08/supremes-slap-bide...   
2  https://www.wnd.com/2021/07/gop-stages-covid-r...   
3  https://www.wnd.com/2021/08/canada-reopens-bor...   
4  https://www.wnd.com/2021/05/vaccine-passports-...   

                                           Image_url  Label  \
0  https://www.wnd.com/wp-content/uploads/2021/04...      1   
1  https://www.wnd.com/wp-content/uploads/2020/10...      1   
2  https://www.wnd.com/wp-content/uploads/2020/05...      0   
3  https://www.wnd.com/wp-content/uploads/2019/07...      1   
4  https://

In [25]:
!pip install git+https://github.com/openai/CLIP.git

import clip
from PIL import Image
import torch

# Set the device
device = "cuda" if torch.cuda.is_available() else "cpu"
clip_model, preprocess = clip.load("ViT-B/32", device=device)


  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-leys5bv1
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-req-build-leys5bv1
  Resolved https://github.com/openai/CLIP.git to commit a1d071733d7111c9c014f024669f959182114e33
  Preparing metadata (setup.py) ... done


This process will provide you with a rich set of features extracted from both the text and images in your dataset, leveraging the power of CLIP.

In [26]:
def get_clip_features(texts, image_paths):
    text_inputs = clip.tokenize(texts).to(device)

    image_inputs = [preprocess(Image.open(path)).unsqueeze(0) for path in image_paths]
    image_inputs = torch.cat(image_inputs).to(device)

    with torch.no_grad():
        text_features = clip_model.encode_text(text_inputs)
        image_features = clip_model.encode_image(image_inputs)

        text_features /= text_features.norm(dim=-1, keepdim=True)
        image_features /= image_features.norm(dim=-1, keepdim=True)

    combined_features = torch.cat((text_features, image_features), dim=1)
    return combined_features.cpu()


In [27]:
batch_size = 32  # Adjust based on your GPU's memory
n = len(df)
all_features = []

for i in range(0, n, batch_size):
    batch_texts = df['Title'][i:i+batch_size].tolist()
    batch_image_paths = df['Image_Path'][i:i+batch_size].tolist()
    batch_features = get_clip_features(batch_texts, batch_image_paths)
    all_features.append(batch_features)

# Concatenate all batch features
all_features = torch.cat(all_features, dim=0)

# Convert to numpy and store in DataFrame (optional)
df['CLIP_Features'] = [feature.numpy() for feature in all_features]


In [28]:
# Saving the features for future use
df.to_pickle('/content/processed_fakenews_data_with_clip_features.pkl')


In [30]:
inputs, labels = inputs.float().to(device), labels.float().to(device)


Convert Input Tensors to float32: Before feeding data into the model, ensure that your input tensors are converted to float32. This is the default data type for most operations in PyTorch: